In [ ]:
import time
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [ ]:
INPUT_CSV = 'scraped_addresses/geocoded_results.csv'
OUTPUT_CSV = 'scraped_addresses/sa_addresses_enriched.csv'
SCREENSHOT_DIR = 'satellite_imagery'
URL = 'https://location.sa.gov.au/viewer/#'

os.makedirs(SCREENSHOT_DIR, exist_ok=True)

In [ ]:
def setup_driver():
    chrome_options = Options()
    # Uncomment below for headless
    # chrome_options.add_argument('--headless=new')
    chrome_options.add_argument('--start-maximized')
    driver = webdriver.Chrome(options=chrome_options)
    return driver

In [ ]:
def wait_and_click(driver, selector, delay=10):
    el = WebDriverWait(driver, delay).until(EC.element_to_be_clickable((By.CSS_SELECTOR, selector)))
    el.click()
    time.sleep(.75)

In [ ]:
def wait_and_click_xpath(driver, selector, delay=10):
    el = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, selector)))
    el.click()
    time.sleep(.75)

In [ ]:
def press_enter(element):
    element.send_keys(Keys.RETURN)
    time.sleep(.75)

In [ ]:
def press_enter_global(driver):
    actions = ActionChains(driver)
    actions.send_keys(Keys.RETURN).perform()  # or Keys.ENTER
    time.sleep(.75)

In [ ]:
def wait_and_type(driver, selector, text, delay=10):
    el = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
    el.clear()
    el.send_keys(text)
    time.sleep(.75)

In [ ]:
def try_extract_element(driver, selector, delay=10):
    try:
        return WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
    except:
        return None

In [ ]:
def click_in_centre(driver):
    body = driver.find_element(By.TAG_NAME, "body")
    actions = ActionChains(driver)
    actions.move_to_element_with_offset(body, body.size['width']//2, body.size['height']//2).click().perform()

In [ ]:
def set_parcel_blue(driver):
    driver.execute_script("""
    var path = document.querySelector('svg #map_graphics_layer > g > path');
    if (path) {
        path.setAttribute('fill', 'rgb(0, 0, 255)');
        path.setAttribute('stroke', 'rgb(0, 0, 255)');
        path.setAttribute('fill-opacity', '1');
        path.setAttribute('stroke-width', '8');
    }
    """)
    time.sleep(0.75)

In [ ]:
def cleanup_ui_1(driver):
    try:
        driver.execute_script("""
            document.querySelector('#map_root > div.esriPopup.esriPopupVisible > div.esriPopupWrapper')?.style.setProperty('display', 'none');
            document.querySelector('#map_root > div.esriPopup.esriPopupVisible > div.outerPointer.left')?.style.setProperty('display', 'none');
            document.querySelector('#map_graphics_layer > image')?.style.setProperty('display', 'none');
        """)
    except Exception:
        pass

In [ ]:
def cleanup_ui_2(driver):
    try:
        driver.execute_script("""

        #map_graphics_layer > image

#map_graphics_layer > g > path
        
            document.querySelector('#map_root > div.esriPopup.esriPopupVisible > div.esriPopupWrapper')?.style.setProperty('display', 'none');
            document.querySelector('#map_root > div.esriPopup.esriPopupVisible > div.outerPointer.left')?.style.setProperty('display', 'none');
        """)
    except Exception:
        pass

In [ ]:
def show_pointer(driver):
    try:
        driver.execute_script("""
            document.querySelector('#map_graphics_layer > image')?.style.setProperty('display', 'block');
        """)
    except Exception:
        pass

In [ ]:
def process_address(driver, row, i):
    full_address = row['candidate_address']
    result = {
        'parcel plan id': None,
        'error': None
    }

    try:
        base_file_name = str(i) + '_' + full_address.replace(' ', '_').replace(',','')
        driver.get(URL)
        time.sleep(5)
        wait_and_click(driver, '#messagingDiv > div > button') # Close button
        print("completed Close button")
        wait_and_click(driver, '#macSplash > div > div > button')  # Start
        print("completed start")

        # Search Address
        wait_and_type(driver, '#locatorInput', full_address)
        time.sleep(1)
        press_enter_global(driver) # click enter/return
        print("enter button pressed")
        time.sleep(2)
        
        # Zoom in
        zoom_selector = '//*[@id="menuDiv"]/label[9]/div/div[1]'
        for _ in range(4):
            wait_and_click_xpath(driver, zoom_selector) # Close the data search
            time.sleep(0.5)
        print("map zoomed in")
        time.sleep(4)
        print("3-second pause to ensure image has loaded")

        # take first screenshot (clean image)
        cleanup_ui_1(driver)
        screenshot_path = os.path.join(SCREENSHOT_DIR, (base_file_name + '_base.png'))
        driver.find_element(By.ID, 'map_gc').screenshot(screenshot_path)

        # Enable Data Layer
        wait_and_click_xpath(driver, '//*[@id="menuDiv"]/label[2]') # Data Search
        print("completed Data Search")
        wait_and_type(driver, '#dataSearchInput', 'Parcel Cadastre') # Type 'Parcel Cadastre'
        print("Typed 'Parcel Cadastre'")
        press_enter_global(driver) #press return
        print("pressed enter (global)")
        wait_and_click_xpath(driver, '//*[@id="dataSearchResultsDiv"]/input') # Click check-box to add data layer
        print("completed checking check box")
        wait_and_click_xpath(driver, '//*[@id="dataSearchMenuDiv"]/h5/span') # Close the data search
        print("completed closing data search")

        # get green dot back
        show_pointer(driver)
        print("show green pointer")

        #click on pointer
        wait_and_click_xpath(driver, '//*[@id="map_graphics_layer"]')
        print("click on pointer")
       
        # click on 'what's here?'
        wait_and_click_xpath(driver, '//*[@id="moreGraphic"]')
        print("click on what's here")

        time.sleep(1)
        
        # Make the parcel blue
        set_parcel_blue(driver)
        print("set parcel blue")

        # Take second screenshot (blue box)
        time.sleep(1)
        cleanup_ui_1(driver)
        screenshot_path = os.path.join(SCREENSHOT_DIR, (base_file_name + '_boxed.png'))
        driver.find_element(By.ID, 'map_gc').screenshot(screenshot_path)

    except Exception as e:
        result['error'] = str(e)

    return result

In [ ]:
def main():
    df = pd.read_csv(INPUT_CSV)
    df['parcel plan id'] = ""
    df['error'] = ""
    
    for i, row in df.iterrows():
        print(f"[{i+1}/{len(df)}] Processing: {row['candidate_address']}")
        driver = setup_driver()
        result = process_address(driver, row, i)

        # Retry once on failure
        if result['error']:
            print(f" - Error: {result['error']}. Retrying...")
            result = process_address(driver, row, i)

        df.at[i, 'error'] = result['error']
        print(" - Done.")

        driver.quit()

    print(f"Completed. Results saved to {OUTPUT_CSV}")

In [ ]:
if __name__ == '__main__':
    main()